In [45]:
!pip install faiss-cpu rank_bm25

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [46]:
!pip install google-generativeai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [47]:
import json
import os
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
import numpy as np
import faiss
import google.generativeai as genai
from rank_bm25 import BM25Okapi
import math
from kaggle_secrets import UserSecretsClient

json_files = [
    '/kaggle/input/legals/legal_1.json',
    '/kaggle/input/legals/legal_2.json',
    '/kaggle/input/legals/legal_3.json'
]

embedding_model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
reranking_model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
gemini_model_name = 'gemini-2.0-flash' 

loaded_embedding_model = None
loaded_reranking_model = None
loaded_gemini_model = None

In [48]:
def load_embedding_model(model_name):
    global loaded_embedding_model
    if loaded_embedding_model is None:
        print(f"\nLoading Embedding model: '{model_name}'...")
        loaded_embedding_model = SentenceTransformer(model_name)
        print("Model loaded successfully.")
    return loaded_embedding_model

def load_reranking_model(model_name):
    global loaded_reranking_model
    if loaded_reranking_model is None:
        print(f"\nLoading Re-ranking model: '{model_name}'...")
        loaded_reranking_model = CrossEncoder(model_name)
        print("Re-ranking model loaded successfully.")
    return loaded_reranking_model

def load_gemini_model():
    global loaded_gemini_model

    try:
         user_secrets = UserSecretsClient()
         google_api_key = user_secrets.get_secret("GOOGLE_API_KEY")
         if not google_api_key:
              print("Gemini API key not found in Kaggle Secrets. Cannot load Gemini model.")
              return None
    except Exception as e:
         print(f"Error accessing Kaggle Secrets: {e}")
         print("Please ensure you are running in a Kaggle notebook and have added GOOGLE_API_KEY to Secrets.")
         return None

    if loaded_gemini_model is None:
        print(f"\nConfiguring Gemini model: '{gemini_model_name}'...")
        genai.configure(api_key=google_api_key)
        loaded_gemini_model = genai.GenerativeModel(gemini_model_name)
        print("Gemini model loaded successfully.")

    return loaded_gemini_model

In [49]:
def embed_legal_chunks(file_paths, model):
    all_chunks = []

    print(f"Starting data processing.")
    print(f"Reading data from {len(file_paths)} JSON files...")

    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, list):
                all_chunks.extend(data)
                print(f"Read {len(data)} chunks from '{file_path}'. Total: {len(all_chunks)}")
            else:
                print(f"Warning: File '{file_path}' does not contain a JSON list. Skipping.")

    if not all_chunks:
        print("No chunks read from the provided files. Returning empty list.")
        return []

    print(f"\nTotal chunks read successfully: {len(all_chunks)}")

    texts_to_embed = []
    original_indices_of_valid_texts = []

    for i, chunk in enumerate(all_chunks):
        text = chunk.get('text')
        if text and isinstance(text, str):
            texts_to_embed.append(text)
            original_indices_of_valid_texts.append(i)

    if not texts_to_embed:
        print("No valid text content found in chunks for embedding. Returning chunks without embeddings.")
        return all_chunks

    print(f"Creating embeddings for {len(texts_to_embed)} valid text snippets...")

    embeddings = model.encode(texts_to_embed, show_progress_bar=True, convert_to_numpy=True)
    print("Embeddings created.")

    chunks_with_embeddings = []
    embedding_idx = 0

    for i in range(len(all_chunks)):
        chunk = all_chunks[i]
        if i in original_indices_of_valid_texts:
             idx_in_valid_list = original_indices_of_valid_texts.index(i)
             chunk['embedding'] = embeddings[idx_in_valid_list].tolist()
             embedding_idx += 1
        else:
             chunk['embedding'] = None

        chunks_with_embeddings.append(chunk)

    print(f"Added 'embedding' field to {embedding_idx} chunks.")
    print(f"--- Processing finished ---")

    return chunks_with_embeddings


In [50]:
class SimpleVectorDatabase:
    def __init__(self):
        self.index = None
        self.documents = []
        self.embedding_dimension = None
        print("SimpleVectorDatabase initialized.")

    def add_documents(self, chunks_with_embeddings):
        print(f"Adding {len(chunks_with_embeddings)} documents to the database...")

        valid_embeddings = []
        valid_documents = [] # Lưu trữ document objects (dict)
        self.original_chunk_indices = [] # Lưu trữ index gốc từ all_chunks

        for i, chunk in enumerate(chunks_with_embeddings):
            if 'embedding' in chunk and chunk['embedding'] is not None:
                valid_embeddings.append(chunk['embedding'])
                # Lưu trữ toàn bộ chunk object hoặc các key cần thiết
                valid_documents.append({
                    'id': chunk.get('id', 'N/A'),
                    'text': chunk.get('text', ''),
                    'metadata': chunk.get('metadata', {})
                })
                # Lưu index của chunk NÀY trong danh sách input chunks_with_embeddings
                # Điều này không hoàn toàn là index gốc từ all_chunks nếu có chunk bị loại bỏ trước đó
                # Để chắc chắn, cần lưu original_indices_of_valid_texts từ embed_legal_chunks
                # Tuy nhiên, với cấu trúc hiện tại, index trong valid_documents TƯƠNG ỨNG với index trong self.index
                # Index trong self.documents cũng tương ứng với index trong self.index
                # Chúng ta cần ánh xạ index từ self.index về index trong danh sách documents gốc
                # Cách đơn giản nhất là lưu document object vào self.documents theo đúng thứ tự thêm vào index.
                # Index trả về từ Faiss sẽ là index trong self.index và self.documents.
                # Điều này ổn.

        if not valid_embeddings:
            print("No valid embeddings found to add to the database.")
            return

        embeddings_array = np.array(valid_embeddings).astype('float32')

        if self.index is None:
            self.embedding_dimension = embeddings_array.shape[1]
            self.index = faiss.IndexFlatL2(self.embedding_dimension)
            print(f"Created Faiss index with dimension: {self.embedding_dimension}")

        if embeddings_array.shape[1] != self.embedding_dimension:
             print(f"Warning: Embedding dimension mismatch. Expected {self.embedding_dimension}, got {embeddings_array.shape[1]}. Skipping adding these embeddings.")
             return

        self.index.add(embeddings_array)
        # self.documents lưu các document object theo thứ tự được thêm vào index
        self.documents.extend(valid_documents)
        print(f"Added {len(valid_embeddings)} embeddings to the index.")
        print(f"Total documents in database: {len(self.documents)}")
        print(f"Total embeddings in index: {self.index.ntotal}")


    def search(self, query_embedding, k=5):
        if self.index is None:
            print("Database is empty. Cannot perform search.")
            return [], []

        if query_embedding is None:
            print("Query embedding is None. Cannot perform search.")
            return [], []

        query_embedding_array = np.array([query_embedding]).astype('float32')

        if query_embedding_array.shape[1] != self.embedding_dimension:
            print(f"Warning: Query embedding dimension mismatch. Expected {self.embedding_dimension}, got {query_embedding_array.shape[1]}. Cannot perform search.")
            return [], []

        print(f"Searching for top {k} similar documents...")
        distances, indices = self.index.search(query_embedding_array, k)
        print("Search complete.")

        # Trả về khoảng cách và các index của tài liệu trong self.documents
        return distances[0], indices[0]

In [51]:
def retrieve_relevant_chunks(query_text, embedding_model, vector_db, k=5):
    """
    Embeds the query text and retrieves the top k relevant chunk indices from the vector database.
    """
    if embedding_model is None:
        print("Embedding model is not loaded. Cannot embed query.")
        return [], []

    if vector_db.index is None:
        print("Vector database is not initialized or empty. Cannot search.")
        return [], []

    print(f"\nEmbedding query text: \"{query_text}\"")
    query_embedding = embedding_model.encode(query_text, convert_to_numpy=True).astype('float32')
    print("Query embedding created.")

    print(f"Searching database for top {k} results...")
    # vector_db.search đã sửa để trả về distances và indices
    distances, indices = vector_db.search(query_embedding, k=k)
    print("Search completed.")

    # Trả về khoảng cách và các index
    return distances, indices


In [56]:
class HybridRetriever:
    def __init__(self, vector_db, documents):
        self.vector_db = vector_db
        self.documents = documents
        self.bm25 = None
        self.document_texts = [doc.get('text', '') for doc in documents]

        print("\nInitializing BM25 index...")
        tokenized_corpus = [self._simple_tokenize(text) for text in self.document_texts]
        self.bm25 = BM25Okapi(tokenized_corpus)
        print("BM25 index initialized.")

    def _simple_tokenize(self, text):
        if not text:
            return []
        text = text.lower()
        text = text.replace('.', '').replace(',', '').replace(';', '').replace(':', '').replace('"', '').replace("'", "").replace('(', '').replace(')', '').replace('[','').replace(']','').replace('{','').replace('}','').replace('-',' ').replace('/',' ')
        return text.split()

    def _rank_fusion_indices(self, rank_lists, k=60):
        fused_scores = {}

        for rank_list in rank_lists:
            # Đảm bảo rank_list là list hoặc numpy array
            # Nếu là numpy array, chuyển về list để enumerate
            if isinstance(rank_list, np.ndarray):
                rank_list = rank_list.tolist()

            for rank, doc_index in enumerate(rank_list):
                rank_ = rank + 1
                fused_scores[doc_index] = fused_scores.get(doc_index, 0) + (1 / (rank_ + k))

        sorted_indices = sorted(fused_scores, key=fused_scores.get, reverse=True)

        return sorted_indices, fused_scores


    def hybrid_search(self, query_text, embedding_model, vector_search_k=20, final_k=10):
        if self.vector_db is None or self.vector_db.index is None:
            print("Vector database is not initialized or empty. Cannot perform hybrid search.")
            return []

        if self.bm25 is None:
             print("BM25 index is not initialized. Cannot perform hybrid search.")
             return []

        # 1. Vector Search (trả về indices - là numpy array)
        print(f"\nPerforming vector search ({vector_search_k} results)...")
        _, vector_search_indices = retrieve_relevant_chunks(
            query_text,
            embedding_model,
            self.vector_db,
            k=vector_search_k
        )
        print("Vector search completed.")

        # 2. BM25 Search (trả về scores cho TẤT CẢ docs, cần lấy top indices)
        print(f"\nPerforming BM25 search...")
        tokenized_query = self._simple_tokenize(query_text)
        bm25_search_indices = []
        if not tokenized_query:
            print("Query could not be tokenized for BM25 search. BM25 results empty.")
        else:
            bm25_scores = self.bm25.get_scores(tokenized_query)

            bm25_scored_indices = [(score, i) for i, score in enumerate(bm25_scores)]

            bm25_scored_indices = [item for item in bm25_scored_indices if item[0] > 0]

            bm25_scored_indices.sort(key=lambda x: x[0], reverse=True)

            # Lấy chỉ các index từ top BM25 results
            bm25_search_indices = [index for score, index in bm25_scored_indices[:vector_search_k]]
            # bm25_search_indices là một list Python

        print(f"BM25 search completed. Found {len(bm25_search_indices)} results with score > 0.")


        # 3. Rank Fusion (RRF)
        print("\nPerforming Rank Fusion (RRF)...")
        rank_lists_to_fuse = []

        # Sửa lỗi ở đây: Kiểm tra độ dài của mảng NumPy
        if len(vector_search_indices) > 0:
            rank_lists_to_fuse.append(vector_search_indices)

        # Kiểm tra độ dài của list Python
        if len(bm25_search_indices) > 0:
             rank_lists_to_fuse.append(bm25_search_indices)

        fused_indices = []
        fused_scores_dict = {}

        if rank_lists_to_fuse:
             fused_indices, fused_scores_dict = self._rank_fusion_indices(
                 rank_lists_to_fuse,
                 k=60
             )
             print(f"Rank Fusion (RRF) completed. Fused {len(fused_indices)} unique documents.")
        else:
             print("No results from either vector or BM25 search to fuse.")


        # 4. Lấy top final_k documents dựa trên indices đã hợp nhất và sắp xếp
        hybrid_results = []
        for idx in fused_indices[:final_k]:
            doc = self.documents[idx]
            doc_with_score = {
                'doc': doc,
                'hybrid_score': fused_scores_dict.get(idx, 0)
            }
            hybrid_results.append(doc_with_score)


        print(f"Hybrid search completed. Returning top {len(hybrid_results)} results.")
        return hybrid_results

In [71]:
def rerank_documents(query_text, retrieved_documents, reranking_model):
    if not retrieved_documents:
        print("No documents to re-rank.")
        return []

    if reranking_model is None:
        print("Re-ranking model is not loaded. Cannot re-rank.")
        return [{"doc": doc, "score": None} for doc in retrieved_documents]

    print(f"\nRe-ranking {len(retrieved_documents)} documents...")

    # retrieved_documents ở đây là list of document_dict (từ kết quả hybrid search)
    sentence_pairs = [[query_text, doc.get('text', '')] for doc in retrieved_documents]

    relevance_scores = reranking_model.predict(sentence_pairs)
    print("Re-ranking scores predicted.")

    scored_documents = []
    # retrieved_documents và relevance_scores có cùng thứ tự
    for i, doc in enumerate(retrieved_documents):
        scored_documents.append({
            'doc': doc, # Lưu trữ toàn bộ thông tin tài liệu gốc (đã có từ hybrid search)
            'score': relevance_scores[i] # Thêm điểm liên quan
        })
        print(scored_documents)
    scored_documents.sort(key=lambda x: x['score'], reverse=True)
    print("Documents re-ranked by relevance score.")

    return scored_documents


In [72]:
def generate_answer_with_gemini(query_text, relevant_documents, gemini_model):
    if gemini_model is None:
        return "Error: Gemini model is not loaded or configured."

    if not relevant_documents:
        print("No relevant documents provided for generation.")
        context_text = "Không có thông tin ngữ cảnh nào được cung cấp."
    else:
        print(f"\nPreparing prompt with {len(relevant_documents)} relevant documents for Gemini...")
        # relevant_documents là danh sách các dictionary từ rerank_documents, mỗi dict có key 'doc' chứa thông tin gốc
        context_parts = [doc_item.get('doc', {}).get('text', '') for doc_item in relevant_documents if doc_item and doc_item.get('doc')]
        # Chỉ lấy các phần context không rỗng
        context_parts = [part for part in context_parts if part]
        context_text = "\n---\n".join(context_parts)

        if not context_text:
             print("No valid text content found in relevant documents for prompt.")
             context_text = "Không có thông tin ngữ cảnh nào được cung cấp."


    prompt = f"""Bạn là một trợ lý giao thông đường bộ Việt Nam hữu ích. Hãy sử dụng các đoạn văn bản sau để trả lời câu hỏi của người dùng. Nếu các đoạn văn bản không chứa thông tin liên quan đến câu hỏi, chỉ cần nói rằng bạn không tìm thấy thông tin phù hợp dựa trên ngữ cảnh được cung cấp. Đừng cố gắng tạo ra câu trả lời từ kiến thức của riêng bạn. 
    Tuy nhiên bạn cũng phải có khả năng liên tưởng những trường hợp đồng nghĩa ví dụ Rượu, bia, Đèn đỏ thì liên quan tới đèn tín hiệu, xe mô tô thì là xe 2 bánh, ... chứ không phải khớp hoàn toàn thì mới được

Đoạn văn bản:
---
{context_text}
---

Câu hỏi: {query_text}

Trả lời:
"""

    print("Sending prompt to Gemini API...")
    # Gọi API Gemini để tạo nội dung
    response = gemini_model.generate_content(prompt)
    print("Gemini API call finished.")

    # Trả về văn bản kết quả
    if hasattr(response, 'text'):
         return response.text
    else:
         print(f"Gemini response did not contain text. Check response: {response}")
         return "Could not generate answer (response lacked text)."

In [73]:
# --- Phần chạy chính của script ---

print("Starting full RAG process with Hybrid Search, Re-ranking and Generation...")

# 1. Load ALL necessary models once
loaded_embedding_model = load_embedding_model(embedding_model_name)
loaded_reranking_model = load_reranking_model(reranking_model_name)
loaded_gemini_model = load_gemini_model() # Load Gemini model (lấy key từ Kaggle Secrets bên trong hàm)

# Kiểm tra nếu model Gemini không load được thì dừng lại
if not loaded_gemini_model:
     print("\nGemini model could not be loaded. Please check your GOOGLE_API_KEY in Kaggle Secrets.")
else:
     # Tiếp tục quy trình RAG nếu model Gemini đã load và các model khác cũng load thành công

    if loaded_embedding_model and loaded_reranking_model:
        # 2. Process data and create embeddings
        processed_data_with_embeddings = embed_legal_chunks(json_files, loaded_embedding_model)

        if processed_data_with_embeddings:
            print("\nProcessing and embedding complete.")
            valid_embedding_count = len([c for c in processed_data_with_embeddings if 'embedding' in c and c['embedding'] is not None])
            print(f"Total chunks with successful embeddings: {valid_embedding_count}/{len(processed_data_with_embeddings)}")

            if valid_embedding_count > 0:
                 # 3. Build Vector Database
                 print("\n--- Building Vector Database ---")
                 vector_db = SimpleVectorDatabase()
                 # Chỉ thêm các chunk có embedding vào DB
                 chunks_with_valid_embeddings = [c for c in processed_data_with_embeddings if 'embedding' in c and c['embedding'] is not None]
                 vector_db.add_documents(chunks_with_valid_embeddings)
                 print("--- Vector Database Built ---")

                 # --- Khởi tạo Hybrid Retriever ---
                 # Truyền danh sách documents đã được thêm vào vector_db (chỉ những cái có embedding và text hợp lệ)
                 hybrid_retriever = HybridRetriever(vector_db, vector_db.documents)
                 print("--- Hybrid Retriever Initialized ---")

                 # 4. Perform Hybrid Search (thay thế bước initial search)
                 sample_query_text = "Nếu uống rượu bia thì phạt như thế nào?"
                 vector_k = 20 # Số lượng kết quả từ vector search đưa vào RRF
                 final_hybrid_k = 15 # Số lượng kết quả sau RRF đưa vào re-ranking

                 print(f"\n--- Performing Hybrid Search for query: \"{sample_query_text}\" (Vector K: {vector_k}, Final Hybrid K: {final_hybrid_k}) ---")
                 # hybrid_search trả về danh sách các dictionary {'doc': doc, 'hybrid_score': score}
                 initial_hybrid_results = hybrid_retriever.hybrid_search(
                     sample_query_text,
                     loaded_embedding_model,
                     vector_search_k=vector_k,
                     final_k=final_hybrid_k
                 )

                 docs_for_reranking = [item['doc'] for item in initial_hybrid_results]


                 # 5. Perform Re-ranking on results from Hybrid Search
                 final_num_results_after_rerank = 10

                 if docs_for_reranking: # Re-rank chỉ khi có kết quả từ search/hybrid ban đầu
                     if loaded_reranking_model:
                         print(f"\n--- Performing Re-ranking on {len(docs_for_reranking)} results from Hybrid Search ---")
                         reranked_results = rerank_documents(
                             sample_query_text,
                             docs_for_reranking,
                             loaded_reranking_model
                         )

                         # reranked_results là list of {'doc': doc_dict, 'score': score}
                         final_relevant_documents = reranked_results[:final_num_results_after_rerank]

                         print(f"\n--- Top {len(final_relevant_documents)} Re-ranked documents selected for Generation ---")

                         # 6. Generate Answer using Gemini
                         print("\n--- Generating Answer using Gemini ---")
                         final_answer = generate_answer_with_gemini(
                             sample_query_text,
                             final_relevant_documents,
                             loaded_gemini_model
                         )

                         print("\n--- Final Generated Answer ---")
                         print(final_answer)
                         print("--- End of Generated Answer ---")

                     else:
                         print("\nRe-ranking model failed to load. Skipping re-ranking and generation.")
                 else:
                     print("\nHybrid search returned no results. Skipping re-ranking and generation.")


            else:
                print("\nNo valid chunks with embeddings were created. Skipping DB build and downstream steps.")
        else:
            print("\nNo data processed successfully from input files.")
    else:
         print("\nOne or more models (Embedding/Re-ranking) failed to load.")


print("\nRAG process finished.")

Starting full RAG process with Hybrid Search, Re-ranking and Generation...
Starting data processing.
Reading data from 3 JSON files...
Read 420 chunks from '/kaggle/input/legals/legal_1.json'. Total: 420
Read 455 chunks from '/kaggle/input/legals/legal_2.json'. Total: 875
Read 935 chunks from '/kaggle/input/legals/legal_3.json'. Total: 1810

Total chunks read successfully: 1810
Creating embeddings for 1810 valid text snippets...


Batches:   0%|          | 0/57 [00:00<?, ?it/s]

Embeddings created.
Added 'embedding' field to 1810 chunks.
--- Processing finished ---

Processing and embedding complete.
Total chunks with successful embeddings: 1810/1810

--- Building Vector Database ---
SimpleVectorDatabase initialized.
Adding 1810 documents to the database...
Created Faiss index with dimension: 768
Added 1810 embeddings to the index.
Total documents in database: 1810
Total embeddings in index: 1810
--- Vector Database Built ---

Initializing BM25 index...
BM25 index initialized.
--- Hybrid Retriever Initialized ---

--- Performing Hybrid Search for query: "Nếu uống rượu bia thì phạt như thế nào?" (Vector K: 20, Final Hybrid K: 15) ---

Performing vector search (20 results)...

Embedding query text: "Nếu uống rượu bia thì phạt như thế nào?"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query embedding created.
Searching database for top 20 results...
Searching for top 20 similar documents...
Search complete.
Search completed.
Vector search completed.

Performing BM25 search...
BM25 search completed. Found 20 results with score > 0.

Performing Rank Fusion (RRF)...
Rank Fusion (RRF) completed. Fused 39 unique documents.
Hybrid search completed. Returning top 15 results.

--- Performing Re-ranking on 15 results from Hybrid Search ---

Re-ranking 15 documents...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Re-ranking scores predicted.
[{'doc': {'id': '168_2024_NĐ-CP_chunk_50', 'text': 'Trường hợp một cá nhân thực hiện nhiều hành vi vi phạm hành chính mà bị xử phạt trong cùng một lần thì bị phạt tiền đối với từng hành vi vi phạm, nếu có hành vi vi phạm bị tước quyền sử dụng giấy phép lái xe và hành vi vi phạm bị trừ điểm giấy phép lái xe thì chỉ áp dụng hình thức tước quyền sử dụng giấy phép lái xe.', 'metadata': {'source': '168_2024_NĐ-CP', 'effective_date': '2024-12-26', 'url': 'https://thuvienphapluat.vn/van-ban/Giao-thong-Van-tai/Nghi-dinh-168-2024-ND-CP-xu-phat-vi-pham-hanh-chinh-an-toan-giao-thong-duong-bo-619502.aspx#'}}, 'score': 0.54926914}]
[{'doc': {'id': '168_2024_NĐ-CP_chunk_50', 'text': 'Trường hợp một cá nhân thực hiện nhiều hành vi vi phạm hành chính mà bị xử phạt trong cùng một lần thì bị phạt tiền đối với từng hành vi vi phạm, nếu có hành vi vi phạm bị tước quyền sử dụng giấy phép lái xe và hành vi vi phạm bị trừ điểm giấy phép lái xe thì chỉ áp dụng hình thức tước quyền